In [ ]:
import os
import json
import requests
import csv
import numpy as np
import pandas as pd
import geopandas as gpd
from zipfile import ZipFile
from datetime import date, datetime, timedelta
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

### Load, extract, and process data

In [ ]:
yearDate = date.today().strftime('%m%d%y')
print(str(datetime.now()) + " Running 5_COVID_Hospital_ICU_Process")
print("today: "+yearDate)
yesterday = date.today() - timedelta(days=1)
yearDate_yesterday = yesterday.strftime('%m%d%y')
print("yesterday: "+yearDate_yesterday)

In [ ]:
dataFile_icu = "data/ICU/ICU_BedsCounty1_data{}.csv".format(yearDate)
dataFile_newhosp = "data/ICU/COVID_Hospitalizations_County1_data{}.csv".format(yearDate)
dataFile_hp = "data/ICU/Hospital_BedsCounty1_data{}.csv".format(yearDate)

In [ ]:
df_ICU = pd.read_csv(dataFile_icu, sep=',',encoding='utf-8')
df_ICU.head()

In [ ]:
df_newhosp = pd.read_csv(dataFile_newhosp,sep=',',encoding='utf-8')
df_newhosp.head()

In [ ]:
df_hp = pd.read_csv(dataFile_hp, sep=',',encoding='utf-8')
df_hp.head()

In [ ]:
df_ICU_pivot = pd.pivot_table(df_ICU,values='Measure Values',index= 'County',columns = 'Measure Names',aggfunc=np.sum)
df_ICU_pivot.head()

In [ ]:
df_hp_pivot = pd.pivot_table(df_hp,values='Measure Values',index= 'County',columns = 'Measure Names',aggfunc=np.sum)
df_hp_pivot.head()

In [ ]:
df_merge = pd.merge(df_ICU_pivot, df_hp_pivot, left_on=['County'],right_on=['County'], how ='outer')
df_merge2 = pd.merge(df_merge, df_newhosp, left_on=['County'],right_on= ['County'], how = 'outer')
df_merge2.head()

In [ ]:
df_merge_sum = df_merge2.append(df_merge2.sum(numeric_only=True), ignore_index=True)

In [ ]:
df_merge_sum['County'] = df_merge_sum['County'].replace(np.nan, 'A State')
df_merge_sum.iat[58,3] = df_merge_sum.iloc[58][2]/df_merge_sum.iloc[58][7]
df_merge_sum.iat[58,5] = df_merge_sum.iloc[58][4]/df_merge_sum.iloc[58][8]
df_merge_sum.iat[58,10] = df_merge_sum.iloc[58][9]/df_merge_sum.iloc[58][12]

In [ ]:
df_merge_sum

In [ ]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [ ]:
df_merge_sum.to_excel(os.path.join(result_folder,"FL_County_Hospital_ICU{}.xlsx".format(yearDate)))

In [ ]:
df_merge_sum['Avail_ICU']=(df_merge_sum['Available Adult ICU'] + df_merge_sum['Available Pediatric ICU'])
df_rename = df_merge_sum.rename(columns={"Adult ICU Census":"AICU_Census", "Available Adult ICU":"AICU_Avail",\
                                      "Available Adult ICU%":"A_ICU", "Pediatric ICU Census":"PICU_Census",\
                                      "Available Pediatric ICU":"PICU_Avail","Available Pediatric ICU%":"P_ICU",\
                                      "Total AdultICU Capacity":"T_AICU", "Total PediatricICU Capacity":"T_PICU",\
                                      "Available":"Bed_Avail", "Available Capacity":"Bed_AP",\
                                      "Total Staffed Bed Capacity":"T_bed", 'COVID Hospitalizations':'New_hosp'})
df_rename.head()

In [ ]:
df_rename['COUNTYNAME'] = df_rename.loc[:, 'County']
df_rename['County']= df_rename['County'].str.title()

In [ ]:
df_rename.loc[df_rename['County']=='A State','County']='State'
df_reset_index = df_rename.reset_index()
df_reset_index.head()

### Generate shapefile

In [ ]:
gdf_county_polygon = gpd.read_file("shp/Florida_Counties.shp")
gdf_county_polygon.head()

In [ ]:
a = set(df_reset_index['County'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

In [ ]:
b.difference(a)

In [ ]:
df_reset_index.loc[df_reset_index['County']=='Miami-Dade','County']='Dade'

In [ ]:
a = set(df_reset_index['County'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

In [ ]:
df_merge_poly = pd.merge(df_reset_index, gdf_county_polygon, left_on = ['County'],right_on = ['COUNTYNAME'], how = 'outer')
df_merge_poly.head()

In [ ]:
df_merge_poly1 = df_merge_poly.drop(columns=['OBJECTID','DEPCODE','COUNTY','DATESTAMP','COUNTYNAME_y','ShapeSTLen','ShapeSTAre'])
df_merge_poly2 = df_merge_poly1.rename(columns={"COUNTYNAME_x": "COUNTYNAME"})
df_merge_poly2.head(59)

In [ ]:
shpfile_name = "Florida_County_Hospital_ICU{}".format(yearDate)
print("Saving {}.shp in results/".format(shpfile_name))
gdf_merge = gpd.GeoDataFrame(df_merge_poly2, geometry='geometry',crs="EPSG:4326")
gdf_merge.to_file(os.path.join(result_folder,"{}.shp".format(shpfile_name)))

In [ ]:
print("Saving {}.zip in publish/".format(shpfile_name))
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))  

### Update the AGOL layer with the shapefile

In [ ]:
# Check if the data is updated or not today
yesterday_covid_file = "data/Florida_COVID19_Cases_by_County{}.xlsx".format(yearDate_yesterday)
today_covid_file = "data/Florida_COVID19_Cases_by_County{}.xlsx".format(yearDate)
df_yester_covid = pd.read_excel(yesterday_covid_file, sheet_name = 'Florida_COVID19_Cases_by_County')
df_today_covid = pd.read_excel(today_covid_file, sheet_name = 'Florida_COVID19_Cases_by_County')
df_yester_newpos = df_yester_covid[['County_1','NewPos']]
df_today_newpos = df_today_covid[['County_1','NewPos']]
df_yester_newpos.head()

In [ ]:
df_today_newpos.head()

In [ ]:
if df_yester_newpos.equals(df_today_newpos): 
    raise Exception("Today's data is not updated yet. Exit the script.")
else: 
    print ("Today's data is updated.") 

In [ ]:
print("Updating Florida COVID Hospital ICU Census layer")
uflURL = "https://ufl.maps.arcgis.com/"
with open("env.json", "r") as f:
    tokenfile = json.load(f)

In [ ]:
my_token = tokenfile['token']
AGOLConnection = GIS(token=my_token)
print(AGOLConnection.users.__dict__)

In [ ]:
hosp_icu_zip_file = "publish/Florida_County_Hospital_ICU{}.zip".format(yearDate)
Hosp_ICU_content = AGOLConnection.content.get('a5e5d4d7be574d11a4a7dcdc1fd5bda2')
Hosp_ICU_content

In [ ]:
Hosp_ICU_collection = FeatureLayerCollection.fromitem(Hosp_ICU_content)
reponse_hosp = Hosp_ICU_collection.manager.overwrite(hosp_icu_zip_file)
print(reponse_hosp)

In [ ]:
print(str(datetime.now()) + " Done with 5_COVID_Hospital_ICU_Process")